SHEN Zhexiao, LACOSTE Lucille, QUILLERIET Marie-Clémentine

# Projet numérique : câble sous-marin

## Partie théorique

### Question 1

Le théorème du cours autorisant l'approximation évoquée est la loi forte des grands nombres.

### Question 2

On prend $Z_{2}=(Z(x_{j_{1}}),...,Z(x_{j_{n}}))$ et $Z_{1}= Z/Z_{2}$
Alors la loi conditionnelle du vecteur des composantes de Z correspondant aux points de discrétisation sans observation, connaissant les valeur prises par les composantes aux sites d'observations est:
    $$f_{Z_{1}|Z_{2}=(z(x_{j_{1}}),...,z(x_{j_{n}}))}=\frac{1}{(2\pi)^{\frac{N-n+2}{2}}}\times\exp(-\frac{1}{2}(z_{1}-\psi(z_{2}))^{t}CS_{z_{2}}^{-1}(z_{1}-\psi(z_{2})))$$
avec :
    $$CS_{Z_{1}}=C_{1}-C_{Z_{1},Z_{2}}C_{Z_{2}}^{-1}C_{Z_{2},Z_{1}}$$
    $$\psi(z_{2})=m_{Z_{1}}+C_{Z_{1},Z_{2}}C_{Z_{2}}^{-1}(z_{2}-m_{Z_{2}})$$
Et on a :
    $$E(Z_{1}|Z_{2})=m_{Z_{1}}+C_{Z_{1},Z_{2}}C_{Z_{2}}^{-1}(Z_{2}-m_{Z_{2}})$$   

### Question 3

On a :
    $$R.Y=[\sum_{k=1}^{p}r_{ik}Y_{k}]_{i\in[1,p]}$$
    $$Z=[\sum_{k=1}^{p}r_{ik}Y_{k}+m_{i}]_{i\in[1,p]}$$
Donc chaque terme de Z sont une combinaison linéaire des variables aléatoires gaussiennes, alors Z est un vecteur gaussien à densité. Il faut juste calculer les covariances, alors:
    $$Cov(Z_{i},Z_{j})=E((Z_{i}-m_{i})(Z_{j}-m_{j}))=E(\sum_{k=1}^{p}r_{ik}Y_{k}\sum_{l=1}^{p}r_{jl}Y_{l})=\sum_{l,k}r_{ik}r_{jl}\times E(Y_{k}Y_{l})$$
Or, quand $k \not= l$,
    $$E(Y_{k}\times Y_{l}))=\int\int y_{1}y_{2}\frac{1}{2\pi}\exp^{-\frac{y_{1}^{2}+y_{2}^{2}}{2}}dy_{1}dy_{2}=\frac{1}{2\pi} \times (\int y\exp^{-\frac{y^{2}}{2}}dy)^{2} = 0 $$
Alors,
    $$Cov(Z_{i},Z_{j})=\sum_{l,k}r_{ik}r_{jl}\times E(Y_{k}^{2})=\sum_{l,k}r_{ik}r_{jl} \times 1$$
Finalement, la matrice de covariance est:
    $$C=R.R^{t}$$

### Question 4

Comme $ Z1|Z2 = m_{Z1|Z2} + RY $, avec $ C_{Z1|Z2} = R^tR $ et $ Y $ un vecteur à composantes suivant des lois gaussiennes centrées réduites indépendantes :

1. on calcule $ R $ et $ m_{Z1|Z2} $ (qui ne varient pas selon la simulation)

2. puis on simule Y. La simulation de Y se fait grâce au fait que toute variable aléatoire $X$ telle que $ X = \sqrt{-2ln(U)}cos(2\pi V) $, avec $U$ et $V$ deux lois uniformes indépendantes sur $]0,1[$, suive une loi normale centrée réduite. Le but est donc d'écrire $Y$ comme $ Y = \sqrt{-2ln(U)}cos(2\pi V) $ et de simuler $U$ et $V$ grâce au module numpy.random.

## Implémentation

In [ ]:
import numpy as np
import numpy.random as rd
import math
import matplotlib.pyplot as plt

A = 0
B = 500
N = 101
Delta = (B-A)/(N-1)
discretization_indexes = np.arange(N)
discretization = discretization_indexes*Delta

mu = -5
a = 50
sigma2 = 12

observation_indexes = np.array([0, 20, 40, 60, 80, 100])
observation_discretization = observation_indexes*Delta
depth = np.array([0, -4, -12.8, -1, -6.5, 0])
unknown_indexes = np.array(list(set(discretization_indexes)-set(observation_indexes)))
unknown_discretization = unknown_indexes*Delta

### Question 1

On calcule la covariance en distinguant selon que la distance donnée est un scalaire ou une matrice.

In [ ]:
def covariance(dist, a, sigma2):
    C = lambda h: sigma2*np.exp(-np.abs(h)/a)
    if type(dist) == np.ndarray:
        n = dist.shape[0]
        m = dist.shape[1]
        Cov = np.zeros((n,m))
        for i in range(n):
            for j in range(m):
                Cov[i,j] = C(dist[i,j])
        return Cov
    else:
        return C(dist)

### Question 2

In [ ]:
def mat_dist(discretization):
    Dist = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            Dist[i,j] = np.abs(discretization[i]-discretization[j])
    return Dist

mat_dist(discretization)

### Question 3

In [ ]:
def covariance_z(discretization, a, sigma2):
    return covariance(mat_dist(discretization), a, sigma2)

covariance_z(discretization, a, sigma2)

### Question 4

Nous avons essayé une première méthode : calculer la covariance de Z puis extraire les sous matrices demandées. Cependant, cela prenait trop de temps donc nous avons décidé de calculer, pour chaque sous matrice, la sous matrice de distance correspondante puis de calculer la sous-matrice elle-même comme précédemment.

In [ ]:
# 1. Covariance(Z2, Z2)

def mat_dist_obs(observation_discretization):
    n = len(observation_discretization)
    Dist = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            Dist[i,j] = np.abs(observation_discretization[i]-observation_discretization[j])
    return Dist

def cov_obs(observation_discretization, a, sigma2):
    return covariance(mat_dist_obs(observation_discretization), a, sigma2)

# 2. Covariance(Z1, Z2)

def mat_dist_inc_obs(observation_discretization, unknown_discretization):
    n = len(unknown_discretization)
    m = len(observation_discretization)
    Dist = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            Dist[i,j] = np.abs(unknown_discretization[i]-observation_discretization[j])
    return Dist

def cov_inc_obs(observation_discretization, unknown_discretization, a, sigma2):
    return covariance(mat_dist_inc_obs(observation_discretization, unknown_discretization), a, sigma2)

# 3. Covariance(Z2, Z1)

def mat_dist_obs_inc(observation_discretization, unknown_discretization):
    n = len(observation_discretization)
    m = len(unknown_discretization)
    Dist = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            Dist[i,j] = np.abs(observation_discretization[i]-unknown_discretization[j])
    return Dist

def cov_obs_inc(observation_discretization, unknown_discretization, a, sigma2):
    return covariance(mat_dist_obs_inc(observation_discretization, unknown_discretization), a, sigma2)

# 4. Covariance(Z1, Z1)

def mat_dist_inc(unknown_discretization):
    n = len(unknown_discretization)
    Dist = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            Dist[i,j] = np.abs(unknown_discretization[i]-unknown_discretization[j])
    return Dist

def cov_inc(unknown_discretization, a, sigma2):
    return covariance(mat_dist_inc(unknown_discretization), a, sigma2)

### Question 5

On utilise la formule du cours:
$ E(Z1|Z2) = m_{Z1} + C_{Z1,Z2}C_{Z2}^{-1}(Z2 - m_{Z2}) $, sachant que selon l'énoncé, les $m_{Z1}$ et $m_{Z2}$ sont des vecteurs constitués uniquement de $\mu$.

In [ ]:
def esp_conditionnelle(observation_indexes, unknown_indexes, observation_discretization, unknown_discretization, depth, a ,sigma2):
    esp_obs = np.array([mu for i in observation_indexes])
    esp_inc = np.array([mu for i in unknown_indexes])
    depth = np.array(depth)
    inv_cov_obs = np.linalg.inv(cov_obs(observation_discretization, a ,sigma2))
    produit1 = np.dot(inv_cov_obs, depth - esp_obs)
    produit2 = np.dot(cov_inc_obs(observation_discretization, unknown_discretization, a ,sigma2), produit1)
    return esp_inc + produit2

### Question 6

On utilise la formule du cours :
$ Cov(Z1|Z2) = C_{Z1} - C_{Z1,Z2}C_{Z2}^{-1}C_{Z2,Z1} $

In [ ]:
def cov_conditionnelle(observation_discretization, unknown_discretization, a ,sigma2): # cov(Z1|Z2)
    C_z1 = cov_inc(unknown_discretization, a, sigma2)
    C_z1_z2 = cov_inc_obs(observation_discretization, unknown_discretization, a ,sigma2)
    C_z2_z1 = cov_obs_inc(observation_discretization, unknown_discretization, a ,sigma2)
    inv_C_z2 = np.linalg.inv(cov_obs(observation_discretization, a ,sigma2))
    produit1 = np.dot(inv_C_z2, C_z2_z1)
    produit2 = np.dot(C_z1_z2, produit1)
    return C_z1 - produit2

n = len(unknown_indexes)
diag = np.array([cov_conditionnelle(observation_discretization, unknown_discretization, a, sigma2)[i,i] for i in range(n)])
plt.scatter(unknown_discretization, diag)
plt.title("Variance conditionnelle")
plt.show()

On voit que la variance conditionnelle fait des "sauts de mouton", en revenant à 0 aux points dont on connait la profondeur. C'est normal car plus on s'approche d'un point dont on connaît la profondeur (qu'on note $Z(x_{j_{i}})$), plus la covariance entre ce point et celui qui s'en approche est grande, i.e. plus les deux variables sont corrélées, à tel point qu'au bout d'un moment la variable aléatoire modélisant la profondeur du point qui s'approche de $Z(x_{j_{i}})$ se comporte presque comme une constante. Or la variance d'une constante est nulle, d'où la cohérence du résultat trouvé. 

La variance conditionnelle est donc maximale lorsque que l'on se place au milieu de deux points consécutifs dont on connait la profondeur, car c'est là que la variable aléatoire modélisant la profondeur est le plus décorrélée par rapport aux deux points dont on connait la profondeur. 

### Question 7

Comme $ Z_1|Z_2 = m_{Z_1|Z_2} + RY $, avec $ C_{Z_1|Z_2} = R^tR $, et $ Y $ un vecteur à composantes suivant des lois gaussiennes centrées réduites indépendantes, on calcule $ R $ et $ m_{Z_1|Z_2} $ (qui ne varient pas selon la simulation), puis on simule Y.

La simulation de Y se fait grâce au fait que $ X = \sqrt{-2ln(U)}cos(2\pi V) $, avec $U$ et $V$ deux lois uniformes indépendantes sur $]0,1[$, suive une loi normale centrée réduite.

In [ ]:
def R(C):
    return np.linalg.cholesky(C)

esp_cond = esp_conditionnelle(observation_indexes, unknown_indexes, observation_discretization, unknown_discretization, depth, a ,sigma2)

In [ ]:
def simulation_loi_cond(unknown_indexes, observation_discretization, unknown_discretization, a ,sigma2):
    n = len(unknown_indexes)
    Y = []
    for i in range(n):
        yi = np.sqrt(-2*np.log(rd.random()))*np.cos(2*np.pi*rd.random())
        Y.append(yi)
    C = cov_conditionnelle(observation_discretization, unknown_discretization, a ,sigma2)
    return esp_cond + np.dot(R(C), Y)

On plotte ensuite le vecteur $ Z_1 $ des $z$ inconnus trouvés par simulation, et le vecteur $ Z_2 $ des $z$ déjà observés. On plotte également l'espérance conditionnelle de $ Z_1 $ sachant $ Z_2 $. Il faut voir si cette espérance est une bonne approximation de $ Z_1 $ trouvé par simulation, tout comme on approche $L$ par son espérance conditionnelle.

In [ ]:
plt.figure()
plt.scatter(unknown_discretization, simulation_loi_cond(unknown_indexes, observation_discretization, unknown_discretization, a ,sigma2))
plt.scatter(observation_discretization, depth)
plt.scatter(unknown_discretization, esp_cond)
plt.legend(["simulation", "z observés", "espérance conditionnelle"])
plt.title("z simulés, données, espérance")
plt.show()

On voit que l'espérance conditionnelle colle avec les points dont on connaît la profondeur, ce qui est rassurant.
Ensuite, on voit que l'allure de la courbe formée par les z simulés est grossièrement la même que celle donnée par l'espérance conditionnelle. On pourra donc, par la suite, approximer $Z$ par l'espérance conditionnelle de $Z_1|Z_2$, complétée par $Z_2$.

### Question 8

Pour calculer la longueur du câble à l'issue de la simulation, on utilise Pythagore :

In [ ]:
def longueur_cable(vect_depth, Delta):
    n = len(vect_depth)
    l = 0
    for i in range(1,n):
        l += np.sqrt((Delta)**2 + (vect_depth[i]-vect_depth[i-1])**2)
    return l

### Question 9

On approxime la longueur $L$ par l'espérance conditionnelle de $L$ sachant $ Z_{obs} $, elle-même approximée par une moyenne basique des longueurs trouvées au cours des 100 simulations.

In [ ]:
def longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 100):
    S = 0
    for k in range(simul):
        Z_inc = simulation_loi_cond(unknown_indexes, observation_discretization, unknown_discretization, a ,sigma2)
        Z = [0 for i in range(N)]
        for i, item in enumerate(observation_indexes):
            Z[item] = depth[i]
        for i, item in enumerate(unknown_indexes):
            Z[item] = Z_inc[i]
        l_k = longueur_cable(Z,Delta)
        S += l_k
    return S/simul

Une autre méthode pour approximer $L$ est d'approximer $Z$ par l'espérance conditionnelle de $ Z_{1} $ sachant $ Z_{2} $, complétée par les $z$ déjà connus. On remonte ensuite à $L$ avec Pythagore.

In [ ]:
def longueur_esp_cond(observation_indexes, unknown_indexes, Delta, N):
    Z = [0 for i in range(N)]
    for i, item in enumerate(observation_indexes):
        Z[item] = depth[i]
    for i, item in enumerate(unknown_indexes):
        Z[item] = esp_cond[i]
    return longueur_cable(Z, Delta)

On effectue alors la comparaison entre les deux $L$ trouvés :

In [ ]:
print(longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 100), longueur_esp_cond(observation_indexes, unknown_indexes, Delta, N))

Il y a une différence non négligeable d'environ $22-23 m$ (selon les simulations) entre les deux longueurs trouvées.

### Question 10

In [ ]:
M_n = []
n = np.arange(1,101,1)
somme = 0
for i in n:
    somme += longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, 1)
    M_n.append(somme/i)
plt.scatter(n, M_n)
plt.title("Moyenne des longueurs des câbles en fonction du nombre de simulations")
plt.show()


La moyenne fluctue lorsque l'on fait un petit nombre de simulations, mais se stabilise ensuite sur de grands nombres de simulations, ce qui est logique car faire un grand nombre de simulation permet normalement d'avoir une moyenne plus représentative.

### Question 11

On affiche l'histogramme des $ l_k $ générés sur $n$ simulations, avec $N$ points de discrétisation.

In [ ]:
def histogramme(observation_indexes, unknown_indexes, observation_discretization, unknown_discretization, Delta, N, n):
    liste_lk = []
    for k in range(n):
        lk = longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 1)
        liste_lk.append(lk)
    plt.bar([f"l_{k}" for k in range(n)], height = liste_lk)
    plt.title("Histogramme des longueurs de câble générées")
    plt.show()

histogramme(observation_indexes, unknown_indexes, observation_discretization, unknown_discretization, Delta, N, 100)

### Question 12

#### Méthode 1

Nous avons laissé cette méthode, mais pensons qu'elle est fausse car elle représente en réalité l'intervalle de confiance à $95\%$ de la $\textit {moyenne}$ des longueurs et non des longueurs elles-mêmes - c'est pourquoi l'intervalle trouvé est si petit.

On  pose $ S_n = \sum \limits_{{k=0}}^n L_k $ et $ M_n = \frac {S_n}{n} $. Comme les $ L_k $ sont indépendants, de même loi (de moyenne $m$ de de variance $\sigma$) par théorème central limite, $ \sqrt{n} \frac {M_n - m}{\sigma} \sim N(0,1) $. Or l'intervalle de confiance à $95 \% $ d'une loi normale centrée réduite est : $ ]-1.96 ; 1.96[ $. On note a = 1.96.

$ Alors : \\
P(\sqrt{n} \frac {M_n - m}{\sigma} \in ]-a ; a [) = 0.95 \\
i.e. P(\frac {M_n - m}{\sigma} \in ]\frac {-a}{\sqrt {n}} ; \frac {a}{\sqrt{n}} [) = 0.95 \\
i.e. P(M_n - m \in ]\frac {-a \sigma}{\sqrt {n}} ; \frac {a \sigma}{\sqrt{n}} [) = 0.95 \\
i.e. P(M_n \in ]\frac {-a \sigma}{\sqrt {n}} + m ; \frac {a \sigma}{\sqrt{n}} + m [) = 0.95 $

Ainsi l'intervalle de confiance à $ 95\% $ est donné par $ ]\frac {-a \sigma}{\sqrt {n}} + m ; \frac {a \sigma}{\sqrt{n}} + m [ $.

On approche $m$ par la moyenne des $L_k$, où chaque $L_k$ est obtenu pendant la $k^{ième}$ simulation.

Pour ce qui est du calcul de $\sigma$ : comme $ \sigma^2 = E((L_k - m)^2) $, pour $k \in [0,n]$ on calcule $(L_k - m)^2$ puis on approxime l'espérance de cette quantité par sa moyenne sur les $n$ simulations réalisées.

In [ ]:
# Calcul des L_k

liste_lk = []
for k in range(100):
    lk = longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 1)
    liste_lk.append(lk)
liste_lk = np.array(liste_lk)

In [ ]:
# Intervalle de confiance

m = sum(liste_lk)/len(liste_lk)
sigma = sum((liste_lk - m)**2)/len(liste_lk)
borne_inf = -0.196*sigma + m
borne_sup = 0.196*sigma + m
print(f"L'intervalle de confiance à 95% est ]{borne_inf}, {borne_sup}[")

#### Méthode 2

Cette fois, on trie la liste de $L_k$ obtenus par simulation et on prend les $2/3^{ièmes}$ valeurs du début puis les $97/98^{ièmes}$ valeurs de fin pour être sûr que seuls $5$ valeurs sur $100$ (donc $5\%$ des valeurs) n'appartiennent pas à cet intervalle.

In [ ]:
liste_lk = sorted(liste_lk)
borne_inf = (liste_lk[2]+liste_lk[3])/2
borne_sup = (liste_lk[96]+liste_lk[97])/2
print(f"L'intervalle de confiance à 95% est ]{borne_inf}, {borne_sup}[.")

### Question 13

On génère la liste des $L_k$ puis on compte le nombre de $L_k$ qui sont supérieurs à  $525m$, et on divise par le nombre de $L_k$ calculés pour avoir la probabilité voulue. 

On fait cette opération 5 fois puis on prend la moyenne des probabilités obtenues.

In [ ]:
liste_pba = []
for i in range(5):
    liste_lk = []
    for k in range(100):
        lk = longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 1)
        liste_lk.append(lk)
    liste_lk = np.array(liste_lk)
    pba = sum(liste_lk > 525)/100
    liste_pba.append(pba)
print(f"La probabilité que la longueur du câble dépasse 525m est de {sum(liste_pba)/5}.")

### Question 14

Nous avons seulement fait le cas de 1000 simulations car le code mettait trop de temps à s'exécuter...nous n'avons pas réussi à trouver d'alternatives plus rapides

Comparaison entre les deux L trouvés (pour 1000 simulations cette fois)

In [ ]:
print(longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 1000), longueur_esp_cond(observation_indexes, unknown_indexes, Delta, N))

Représentation de la suite $M_n$ des moyennes des longueurs des câbles en fonction du nombre de simulations (jusqu'à 1000).

In [ ]:
M_n = []
n = np.arange(1,1001,1)
sum = 0
for i in n:
    sum += longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, 1)
    M_n.append(sum/i)
plt.scatter(n, M_n)
plt.title("Moyenne des longueurs des câbles en fonction du nombre de simulations")
plt.show()

Représentation de l'histogramme des longueurs de câble générées sur 1000 simulations.

In [ ]:
histogramme(observation_indexes, unknown_indexes, observation_discretization, unknown_discretization, Delta, N, 1000)

Calcul de l'intervalle de confiance à $95\%$ de la longueur du câble, pour 1000 simulations.

In [ ]:
# Calcul des L_k

liste_lk = []
for k in range(1000):
    lk = longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 1)
    liste_lk.append(lk)
liste_lk = np.array(liste_lk)

# Intervalle de confiance

liste_lk = sorted(liste_lk)
borne_inf = (liste_lk[2]+liste_lk[3])/2
borne_sup = (liste_lk[996]+liste_lk[997])/2
print(f"L'intervalle de confiance à 95% est ]{borne_inf}, {borne_sup}[")

Calcul de l'estimation de la probabilité que la longueur du câble dépasse 525m.

In [ ]:
liste_pba = []
for i in range(5):
    liste_lk = []
    for k in range(1000):
        lk = longueur_cable_simul(observation_indexes, unknown_indexes, Delta, N, observation_discretization, unknown_discretization, a ,sigma2, simul = 1)
        liste_lk.append(lk)
    liste_lk = np.array(liste_lk)
    pba = sum(liste_lk > 525)/1000
    liste_pba.append(pba)
print(f"La probabilité que la longueur du câble dépasse 525m est de {sum(liste_pba)/5}.")